In [7]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.core.display import display, HTML
# make sure pandas doesn't truncate the dataframe when rendered
pd.set_option('display.max_rows', 500)

import tasking_manager_client as tm

# set the text search keyword
textSearch = 'covid'
projectStatuses = 'ARCHIVED,PUBLISHED,DRAFT'

# for notebooks used for reporting, it's nice to
# timestamp of the data, since it can get old quickly
now = datetime.utcnow()
print(f'data last updated: {now} UTC')
print(f'tasking manager:   {tm.DEFAULT_INSTANCE_API}')
print(f'search keyword:    {textSearch}')

data last updated: 2020-07-16 11:54:09.319969 UTC
tasking manager:   tasking-manager-tm4-production-api.hotosm.org
search keyword:    covid


In [6]:
   
# concatenate paginated API results into one dataframe
result = pd.concat(tm.v2.project_search(textSearch=textSearch, projectStatuses=projectStatuses), ignore_index=True);
# search API pagination sometimes returns the same project more than once (due to pageintaion cacheing)
result = result.drop_duplicates(subset='projectId');

In [10]:
result.columns

Index(['activeMappers', 'campaigns', 'country', 'dueDate', 'lastUpdated',
       'locale', 'mapperLevel', 'name', 'organisationLogo', 'organisationName',
       'percentMapped', 'percentValidated', 'priority', 'projectId',
       'shortDescription', 'status', 'totalContributors', 'page'],
      dtype='object')

In [57]:
result['country'] = result['country'].apply(lambda val: val[0] if isinstance(val, list) else val)
display_result = result.sort_values(by=['country', 'status','projectId' ])
display_result = display_result[['projectId','country','percentValidated', 'percentMapped', 'mapperLevel','status', 'priority']]
display_result.to_csv('covid-textSearch.csv')


In [62]:
styled = display_result.style\
    .background_gradient(cmap=tm.style.hotosm_cmap(),subset=['percentMapped', 'percentValidated'])\
    .applymap(tm.style.format_priority, subset=['priority'])\
    .applymap(tm.style.format_status, subset=['status'])
styled

,projectId,country,percentValidated,percentMapped,mapperLevel,status,priority
188,8512,Botswana,100,100,BEGINNER,ARCHIVED,LOW
196,8571,Botswana,99,99,BEGINNER,ARCHIVED,LOW
66,8572,Botswana,100,100,BEGINNER,ARCHIVED,MEDIUM
197,8573,Botswana,100,100,BEGINNER,ARCHIVED,LOW
198,8574,Botswana,99,99,BEGINNER,ARCHIVED,LOW
199,8575,Botswana,99,99,BEGINNER,ARCHIVED,LOW
200,8576,Botswana,100,100,BEGINNER,ARCHIVED,LOW
201,8577,Botswana,100,100,BEGINNER,ARCHIVED,LOW
202,8578,Botswana,98,100,BEGINNER,ARCHIVED,LOW
203,8579,Botswana,93,98,BEGINNER,ARCHIVED,LOW
